## Summary

## Library and modules

In [ ]:
import numpy as np, pandas as pd, geopandas as gpd, sklearn.metrics as metrics
import geemap, ee , rasterio, sankee, warnings, pickle
from rasterio.plot import show
from geopandas import GeoDataFrame
from osgeo import gdal, osr, gdalconst
import os, glob, subprocess
import rasterio
import matplotlib as mpl
import matplotlib.pyplot as plt, plotly.express as px
from geemap import ml
from sklearn import ensemble
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
from shapely.geometry import Polygon, Point
from sklearn.metrics import confusion_matrix, cohen_kappa_score
warnings.filterwarnings('ignore')

### Earth Engine log in

In [ ]:
#ee.Authenticate()
ee.Initialize()

In [2]:
#Bit Attribute
#- 0 Fill
#- 1 Clear
#- 2 Water
#- 3 Cloud Shadow
#- 4 Snow
#- 5 Cloud
#- 6-7 Cloud Confidence (00 = None, 01 = Low, 10 = Medium, 11 = High)
#- 8-9 Cirrus Confidence (00 = None, 01 = Low, 10 = Medium, 11 = High)
#- 10 Terrain Occlusion

def maskL8clouds(image):
    cloud_mask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 3)).eq(0) 
    snow_mask  =  image.select('QA_PIXEL').bitwiseAnd(int('11111', 4)).eq(0)
    shadow_mask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 5)).eq(0)

    opticalBands = image.select('B.').multiply(0.0001)
    return (image.addBands(opticalBands, None, True) 
                .updateMask(cloud_mask)  
                .updateMask(snow_mask) 
                .updateMask(shadow_mask)  
           )


In [ ]:

L8 = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(coastal_waters)
    .filterMetadata('CLOUD_COVER', 'less_than', 10)
    .filterDate(start_date, end_date)
    .map(maskL8clouds)
)